In [1]:
from source.class_yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer
yolo_trainer = YOLOTrainer()

In [ ]:
yolo_trainer.image_folder = "PPE_CAPACETE/images"
yolo_trainer.annotations_folder = "PPE_CAPACETE/labels"
yolo_trainer.yolo_Classes = "Hardhat"
yolo_trainer.test_percentual_divisor = 20
yolo_trainer.slicing()

In [2]:
yolo_trainer.img_sz = 960
yolo_trainer.training_epochs = 20
yolo_trainer.task = "detect"
yolo_trainer.training()

New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.79  Python-3.12.1 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=ppe.pt, data=D:/Judson_projetos/Yolo_trainer/YOLO_tools/dataset/dataset_YOLO/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, class

c:\Users\AUTOU4\.virtualenvs\Yolo_trainer-_CygeRZP\Lib\site-packages\ultralytics\engine\trainer.py:271: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning D:\Judson_projetos\Yolo_trainer\YOLO_tools\dataset\dataset_YOLO\labels\train.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
val: Scanning D:\Judson_projetos\Yolo_trainer\YOLO_tools\dataset\dataset_YOLO\labels\val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]


Plotting labels to runs\detect\train25\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 960 train, 960 val
Using 8 dataloader workers
Logging results to runs\detect\train25
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      4.51G      1.401      3.547      1.796         16        960: 100%|██████████| 2/2 [00:07<00:00,  3.96s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.05it/s]

                   all          5          5    0.00733          1     0.0244     0.0152


...

      20/20       4.5G     0.7643      2.069      1.306          7        960: 100%|██████████| 2/2 [00:00<00:00,  5.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.58it/s]

                   all          5          5      0.988          1      0.995      0.805



20 epochs completed in 0.011 hours.
Optimizer stripped from runs\detect\train25\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train25\weights\best.pt, 6.3MB

Validating runs\detect\train25\weights\best.pt...
Ultralytics YOLOv8.2.79  Python-3.12.1 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 19.49it/s]


                   all          5          5      0.988          1      0.995      0.805
                Person          5          5      0.988          1      0.995      0.805
Speed: 0.4ms preprocess, 2.2ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs\detect\train25


In [ ]:
yolo_trainer.train_model = "train16"
yolo_trainer.object_to_predict = "D:/Judson_projetos/Yolo_trainer/2024-07-08_12.15.50.png"
yolo_trainer.predict_confidence = 0.2
yolo_trainer.predict()

In [ ]:
from utils.check_file import check_file

check_file("any_file")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("PPE_CAPACETE", "PPE_CAPACETE")